In [5]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

train=[]
test=[]
check=[]

with open('rating.csv') as fil:
    for line in fil:
        #data.append(line)
        x=random.randint(1,100)
        if x >30:
            train.append(line)
        else:
            test.append(line)
        if x>99:
            check.append(line)
#train, test = train_test_split(data, test_size=0.3)


In [6]:
animeid=[]
userid=[]
rating=[]

print len(train)
print len(test)
print len(check)

for line in check:
    data=line.split(',')
    animeid.append(int(data[1]))
    userid.append(int(data[0]))
    rating.append(float(data[2]))
    
usercount = set(userid)
usercount = list(usercount) #List of users from train set

anime = set(animeid)
anime = list(anime)   #List of anime from train set

5470978
2342759
77661


In [7]:
from collections import defaultdict
import math

ratings_matrix=defaultdict(dict)  

users=[]

for i in range(10):
    users.append(usercount[i])

for i in usercount:
    for j in anime:
        ratings_matrix[i][j]=-1
        
for i in range(len(userid)):
    #if userid[i] in users:
    ratings_matrix[userid[i]][animeid[i]]=rating[i]

In [9]:
user_avg = dict()
rating_count = dict()

for i in usercount:
    for j in anime:
        if i in user_avg:
            if ratings_matrix[i][j] != -1:
                user_avg[i]+=ratings_matrix[i][j]
                rating_count[i]+=1
        else:
            if ratings_matrix[i][j] != -1:
                user_avg[i]=ratings_matrix[i][j]
                rating_count[i]=1
    if i in user_avg:
        user_avg[i] = user_avg[i]/rating_count[i]
    
'''for i in users:
    for j in anime:
        if ratings_matrix[i][j] != 0:
            ratings_matrix[i][j] -= user_avg[i]'''
        
    
user_sim=defaultdict(dict)

for i in users:
    for j in usercount:
        user_sim[i][j]=0

norm1=0
norm2=0
for i in users:
    for j in usercount:
        user_sim[i][j]=0        
        for k in anime:
            if ratings_matrix[i][k]!=-1 and ratings_matrix[j][k]!=-1:
                user_sim[i][j]+=(ratings_matrix[i][k]-user_avg[i])*(ratings_matrix[j][k]-user_avg[j])
                norm1+=(ratings_matrix[i][k]-user_avg[i])*(ratings_matrix[i][k]-user_avg[i])
                norm2+=(ratings_matrix[j][k]-user_avg[j])*(ratings_matrix[j][k]-user_avg[j])

        norm1 = math.sqrt(norm1*norm2)
        if norm1!=0:
            user_sim[i][j] = user_sim[i][j]/norm1 

In [11]:
# kNN
k_user=defaultdict(dict)
k=len(usercount)/10
user_big=0

for i in users:
    for m in range(k):
        big=-99.99
        for j in usercount:
            if user_sim[i][j] > big:
                big = user_sim[i][j]
                user_big = j
        #print user_big, ' ', m 
        k_user[i][m]=user_big
        #print user_sim[i][user_big],' 1 '
        user_sim[i][user_big]-=100
        #print user_sim[i][user_big], ' 2 '
        #print
        
    for m in range(k):
        user_sim[i][k_user[i][m]]+=100

In [28]:
pred_rating=defaultdict(dict)

for i in users:
    for m in anime:
        pred_rating[i][m]=0
        for j in range(k):
            if ratings_matrix[k_user[i][j]][m] != 0:
                pred_rating[i][m] += 0.1*user_sim[i][k_user[i][j]]*ratings_matrix[k_user[i][j]][m]
        if i in user_avg:
            pred_rating[i][m]+=user_avg[i]

In [29]:
tanimeid=[]
tuserid=[]
trating=[]
error_diff=defaultdict(dict)
count=0
checktest=[]
with open('rating.csv') as fil:
    for line in fil:
        #data.append(line)
        x=random.randint(1,100)
        if x >99:
            checktest.append(line)
            
for line in checktest:
    data = line.split(',')
    tanimeid.append(int(data[1]))
    tuserid.append(int(data[0]))
    trating.append(float(data[2]))
    
tanime = set(tanimeid)
tanime = list(tanime)

tuser = set(tuserid)
tuser = list(tuser)

        
tratings_matrix=defaultdict(dict)

for i in usercount:
    for j in anime:
        tratings_matrix[i][j]=0

for i in range(len(tuserid)):
    tratings_matrix[tuserid[i]][tanimeid[i]]=trating[i]
    
for i in users:
    for j in anime:
        error_diff[i][j]=0
        if i in tuser and j in tanime and tratings_matrix[i][j]!=0:
            error_diff[i][j]=abs(tratings_matrix[i][j]-pred_rating[i][j])
            count+=1

In [30]:
# Mean Absolute Error
error_sum=0.0

for i in users:
    for j in anime:
        error_sum+=error_diff[i][j]
mae = error_sum/count
print 'Mean Absolute Error = ', mae

Mean Absolute Error =  2.05934856668


In [31]:
# Root Mean Squared Error
import math

rmse = error_sum*error_sum
rmse = rmse/count
rmse = math.sqrt(rmse)
print 'Root Mean Squared Error = ', rmse

Root Mean Squared Error =  9.43712068911
